In [2]:
! pip install -q datasets transformers seqeval
!pip install -q corus razdel

In [3]:
import pandas as pd
import numpy as np
from datasets import load_dataset, load_metric, Dataset, DatasetDict
from razdel import tokenize
import torch
from sklearn.model_selection import train_test_split


In [4]:
model_checkpoint = "cointegrated/rubert-tiny2"
batch_size = 2

In [5]:
!wget -q -O train.csv https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/fmWGQJvwU5ejog
!wget -q -O train_labels.json https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/0nJ2QTRb9-U7tA

In [6]:
train_df = pd.read_csv('train.csv')
train_labels = pd.read_json('train_labels.json').T

In [7]:
def extract_labels(train_index):
    text = train_df.text[train_index]
    raw_toks = list(tokenize(text))
    words = [tok.text for tok in raw_toks]

    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(text)

    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    spans = train_labels.span[train_index]
    for e in spans:
        entity_type = "ADR"
        e_words = sorted({idx for idx in char2word[e[0]:e[1]] if idx is not None})
        word_labels[e_words[0]] = 'B-' + entity_type
        for idx in e_words[1:]:
            word_labels[idx] = 'I-' + entity_type

    return {'tokens': words, 'tags': word_labels}

In [205]:
print(extract_labels(2))

{'tokens': ['Молоко', 'течёт', 'само', 'у', 'коровы', '.', 'Что', 'делать', ',', 'если', 'у', 'коровы', 'слабые', 'соски', 'и', 'молоко', 'бежит', 'из', 'них', 'само', '.', 'Молоко', 'течёт', 'само', 'у', 'коровы', '.', 'Помогите', 'советом', '.', 'Купили', 'корову', ',', 'было', '2', 'отёла', '.', 'Третий', 'должен', 'быть', 'в', 'феврале', '.', 'Ещё', 'доится', '.', 'Утром', 'и', 'вечером', 'по', '2–3', 'литра', '.', 'Иногда', 'вижу', 'лужицы', 'на', 'месте', ',', 'где', 'полежит', '.', 'Когда', 'доишь', ',', 'молоко', 'порой', 'само', 'бежит', 'из', 'соска', ',', 'который', 'не', 'трогаешь', '.', 'Понятно', ',', 'что', 'соски', 'слабые', '.', 'Бывшая', 'хозяйка', 'говорила', ',', 'что', ',', 'когда', 'с', 'поля', 'возвращается', ',', 'молоко', 'немного', 'вытекает', '.', 'Но', 'это', 'летом', ',', 'с', 'полным', 'выменем', ',', 'на', 'сочных', 'кормах', '.', 'Сейчас', 'не', 'такой', 'случай', '.', 'В', 'интернете', 'везде', 'написано', '—', 'применить', 'коллодий', '.', 'Но', 'его',

In [36]:
ner_data = [extract_labels(i) for i in range(0, 35)]
ner_train, ner_test = train_test_split(ner_data, test_size=0.3, random_state=1)
# ner_train = ner_data

In [37]:
pd.options.display.max_colwidth = 300
pd.DataFrame(ner_train).sample(3)

,tokens,tags
18,"[Как, остановить, понос, у, двухнедельного, телёнка, ., Подскажите, пожалуйста, ,, как, остановить, у, 2-х, недельного, телёнка, понос, ?, Колола, Диспаркол, на, 2–3, дня, прекратился, ,, потом, опять, начался, .]","[O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O]"
10,"[Корова, поносит, и, худеет, апитит, вроде, средний, жвачка, есть, в, чем, может, быть, дело]","[O, B-ADR, O, B-ADR, O, O, O, O, O, O, O, O, O, O]"
7,"[Послеродовый, отёк, вымени, Как, способствовать, устранению, небольшого, отёка, вымени, после, отёла, ?]","[O, B-ADR, I-ADR, O, O, O, O, B-ADR, I-ADR, O, O, O]"


In [38]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 24
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 11
    })
})

In [39]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O', 'B-ADR', 'I-ADR']

In [40]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

loading file https://huggingface.co/cointegrated/rubert-tiny2/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/16af2afaa4ceaa8d50b689bd4c2f7ef7fe3bfac06c0aac7d82a5c1c72298b62a.cc3312d07ccf88871a3c2b7cb3442138e6785101efead94d9f77e96301cf7f4a
loading file https://huggingface.co/cointegrated/rubert-tiny2/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/cfa6d82dc8ecc7fe3f06deb449f38968cdd188bb84c8da0e06f0bbfddbede1e3.550ab7157d36210bf96c7c3b30e621933d37d635c5f2e290f7e88bd5f7c9198a
loading file https://huggingface.co/cointegrated/rubert-tiny2/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/cointegrated/rubert-tiny2/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/20317640533199c6b37a557395cd5ee5fcb8777be7c89bb1314bfd43058b35e9.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/cointegrated/rubert-tiny2/resolv

In [41]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [42]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [61]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

loading configuration file https://huggingface.co/cointegrated/rubert-tiny2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/726139048d10597682731a7a4c0b8ef0382927911bc0f6f050a4f7f0afb04c2a.149cdc07694f3925e290abc5528c57a543bcbc9af955c0202b1028584ad15cb4
Model config BertConfig {
  "_name_or_path": "/gd/MyDrive/models/rubert-tiny-mlm-nli-sentence",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "emb_size": 312,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 600,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 2048,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "pad_token_id": 0

In [62]:
# args = TrainingArguments(
#     "ner",
#     evaluation_strategy = "epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     num_train_epochs=10,
#     weight_decay=0.01,
#     save_strategy='no',
#     report_to='none',
# )

In [63]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [64]:
metric = load_metric("seqeval")

In [65]:
# example = ner_train[4]
# labels = example['tags']
# metric.compute(predictions=[labels], references=[labels])

In [66]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [67]:
# trainer = Trainer(
#     model,
#     args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["test"],
#     data_collator=data_collator,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )

In [68]:
# trainer.evaluate()

In [69]:
# for param in model.bert.parameters():
    # param.requires_grad = False

In [70]:
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name)
#         print(param)

In [71]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [72]:
# trainer.train()

In [73]:
#разморозка
for param in model.parameters():
    param.requires_grad = True

In [74]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

PyTorch: setting up devices


In [75]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [76]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.452799,1.000000,0.033333,0.064516,0.866250
2,No log,0.445475,0.235294,0.400000,0.296296,0.842500
3,No log,0.508215,0.565217,0.216667,0.313253,0.878750
4,No log,0.450561,0.425000,0.283333,0.340000,0.883750
5,No log,0.603758,0.378378,0.233333,0.288660,0.871250
6,No log,0.577641,0.315789,0.300000,0.307692,0.873750
7,No log,0.665071,0.483871,0.250000,0.329670,0.885000
8,No log,0.662034,0.439024,0.300000,0.356436,0.885000
9,No log,0.668572,0.413043,0.316667,0.358491,0.888750
10,No log,0.765067,0.485714,0.283333,0.357895,0.886250


TrainOutput(global_step=240, training_loss=0.08319616317749023, metrics={'train_runtime': 10.8525, 'train_samples_per_second': 44.229, 'train_steps_per_second': 22.115, 'total_flos': 632923639380.0, 'train_loss': 0.08319616317749023, 'epoch': 20.0})

In [77]:
trainer.evaluate()

{'epoch': 20.0,
 'eval_accuracy': 0.88625,
 'eval_f1': 0.36000000000000004,
 'eval_loss': 0.8239219784736633,
 'eval_precision': 0.45,
 'eval_recall': 0.3,
 'eval_runtime': 0.0954,
 'eval_samples_per_second': 115.262,
 'eval_steps_per_second': 62.87}

In [78]:
# predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
# predictions = np.argmax(predictions, axis=2)

# # Remove ignored index (special tokens)
# true_predictions = [
#     [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
#     for prediction, label in zip(predictions, labels)
# ]
# true_labels = [
#     [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
#     for prediction, label in zip(predictions, labels)
# ]

# results = metric.compute(predictions=true_predictions, references=true_labels)
# results

In [79]:
# tokens = tokenizer(text, return_tensors='pt')
# tokens = {k: v.to(model.device) for k, v in tokens.items()}

# with torch.no_grad():
#     pred = model(**tokens)
# pred.logits.shape

In [80]:
# indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
# token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
# for t, idx in zip(token_text, indices):
#     print(f'{t:15s} {label_list[idx]:10s}')

In [81]:
model.save_pretrained('ner_bert/')
tokenizer.save_pretrained('ner_bert/')

Configuration saved in ner_bert/config.json
Model weights saved in ner_bert/pytorch_model.bin
tokenizer config file saved in ner_bert/tokenizer_config.json
Special tokens file saved in ner_bert/special_tokens_map.json


('ner_bert/tokenizer_config.json',
 'ner_bert/special_tokens_map.json',
 'ner_bert/vocab.txt',
 'ner_bert/added_tokens.json',
 'ner_bert/tokenizer.json')

In [82]:
from transformers import pipeline
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device=0)

In [83]:
!wget -q -O test.csv https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/Wo70d4_PAwujqA

In [84]:
test_df = pd.read_csv('test.csv')

In [88]:
test_df

,text_id,text
0,294,"Понос у месячных телят. Подскажите методы и способы лечения жидкого стула у месячных телят. Хватают в рот и лижут всё что видят, сено, солому, лижут соль. Потом страдают. И мы вместе с ними."
1,295,"Понос у телят, чем лечить? \nЧем можно вылечить понос 1,5 месячного телёнка в домашних условиях?\nВетврача у нас нет! Есть лекарство Детрим. Но может быть ещё что-то необходимо? Телёнок месяц сосал корову, потом пил молоко (2 литра за раз).\nВ чём причина? И сколько нужно давать пойла? Только мо..."
2,296,"По какой причине у телёнка отнимаются ноги?\nПодскажите пожалуйста, почему у телёнка ноги отнимаются? Возраст уже полгода, аппетит есть, не поносит, а встать сам не может - вот поднимем, поставим – стоит, но ног как будто не чувствует."
3,297,"Срочно! Ребятки, помогите, корову что-то укусило! У коровы недомогание, вялость! Какая первая помощь? Ветеринар далеко, будет утром, у нас уже 20-00 вечера!"
4,298,"Сгустки у коровы.\nЗдравствуйте, помогите пожалуйста, у меня недавно корова отелилась. 20 дней нормально доила, последние дни замечаю сгустки крови у коровы в молоке. В интернете смотрела — нечего не нашла, кто-то сказал попробовать хозяйственным мылом, массаж делаю — не знаю, что будет.\nДо это..."
...,...,...
94,388,"Большой бык уже неделю поносит, чем помочь?\nПодскажите, пожалуйста, у меня большой бык уже неделю поносит, чем можно помочь?"
95,389,"Как запустить желудок у коровы?\nУ меня объелась корова дерти, залили сразу масло растительное с водой, на следующий день с утра корова ела, а к вечеру запоносила, но не так как хотелось бы и перестала есть. И вот сегодня укололи чемерицу + глауберову соль влили, реакции ноль, при том что ещё бе..."
96,390,"Здрав твуйте, искал похожий случай в темах форума неналёл, такой вопрос купили телят содержатся в одной клетке 5 голов на привязи, у одного телёнка сегодня с утра увидел что он пропоносился какойто прозрачной слизью с какимито белыми хлопьями (наверно овсянка которую ел) запах очень неприятный н..."
97,391,"После отела у коровы ,не отошел послед и было очень твердое вымя.Лечили различными мазями,остались два уплотнения на передних долях вымени.Ни температуры ни сгустков не было.Спустя\n два месяца решили полечить,ввели септогель в доли. Теперь вымя твердое, доит мало, идут сгустки."


In [107]:
ner_model = NER('ner_bert', 'ner_bert')

loading configuration file ner_bert/config.json
Model config BertConfig {
  "_name_or_path": "cointegrated/rubert-tiny2",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "emb_size": 312,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "id2label": {
    "0": "O",
    "1": "B-ADR",
    "2": "I-ADR"
  },
  "initializer_range": 0.02,
  "intermediate_size": 600,
  "label2id": {
    "B-ADR": 1,
    "I-ADR": 2,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 2048,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 83828
}

loading weights file ner_bert/pytorch_model.bin
All model checkpoint weights were used when 

In [108]:
ner_model.hugginface_pipeline(test_df['text'][24])

[{'end': 41,
  'entity_group': 'ADR',
  'score': 0.86486876,
  'start': 32,
  'word': 'по вымени'},
 {'end': 82,
  'entity_group': 'ADR',
  'score': 0.99232185,
  'start': 80,
  'word': 'по'},
 {'end': 89,
  'entity_group': 'ADR',
  'score': 0.96634835,
  'start': 83,
  'word': 'задней'},
 {'end': 99,
  'entity_group': 'ADR',
  'score': 0.56659746,
  'start': 95,
  'word': 'вымя'},
 {'end': 133,
  'entity_group': 'ADR',
  'score': 0.96561325,
  'start': 128,
  'word': 'шишка'},
 {'end': 178,
  'entity_group': 'ADR',
  'score': 0.53499675,
  'start': 173,
  'word': 'шишки'}]

In [109]:
test_df['span'] = test_df['text'].apply(lambda x: ner_model.predict_spans(x))

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:910: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


In [111]:
test_df

,text_id,text,span
0,294,"Понос у месячных телят. Подскажите методы и способы лечения жидкого стула у месячных телят. Хватают в рот и лижут всё что видят, сено, солому, лижут соль. Потом страдают. И мы вместе с ними.","[[68, 72]]"
1,295,"Понос у телят, чем лечить? \nЧем можно вылечить понос 1,5 месячного телёнка в домашних условиях?\nВетврача у нас нет! Есть лекарство Детрим. Но может быть ещё что-то необходимо? Телёнок месяц сосал корову, потом пил молоко (2 литра за раз).\nВ чём причина? И сколько нужно давать пойла? Только мо...","[[524, 530], [372, 378], [538, 544], [753, 757], [142, 147], [484, 489], [214, 220], [291, 297], [752, 757], [47, 52], [339, 344], [364, 369], [412, 417], [467, 472], [558, 563], [593, 598], [785, 790], [785, 791], [558, 565], [558, 566], [553, 557], [531, 535]]"
2,296,"По какой причине у телёнка отнимаются ноги?\nПодскажите пожалуйста, почему у телёнка ноги отнимаются? Возраст уже полгода, аппетит есть, не поносит, а встать сам не может - вот поднимем, поставим – стоит, но ног как будто не чувствует.","[[215, 220], [130, 134], [164, 169], [139, 144], [139, 146]]"
3,297,"Срочно! Ребятки, помогите, корову что-то укусило! У коровы недомогание, вялость! Какая первая помощь? Ветеринар далеко, будет утром, у нас уже 20-00 вечера!","[[72, 76]]"
4,298,"Сгустки у коровы.\nЗдравствуйте, помогите пожалуйста, у меня недавно корова отелилась. 20 дней нормально доила, последние дни замечаю сгустки крови у коровы в молоке. В интернете смотрела — нечего не нашла, кто-то сказал попробовать хозяйственным мылом, массаж делаю — не знаю, что будет.\nДо это...","[[189, 195]]"
...,...,...,...
94,388,"Большой бык уже неделю поносит, чем помочь?\nПодскажите, пожалуйста, у меня большой бык уже неделю поносит, чем можно помочь?","[[23, 28], [98, 103], [23, 30], [98, 105]]"
95,389,"Как запустить желудок у коровы?\nУ меня объелась корова дерти, залили сразу масло растительное с водой, на следующий день с утра корова ела, а к вечеру запоносила, но не так как хотелось бы и перестала есть. И вот сегодня укололи чемерицу + глауберову соль влили, реакции ноль, при том что ещё бе...","[[201, 205], [153, 158], [195, 199]]"
96,390,"Здрав твуйте, искал похожий случай в темах форума неналёл, такой вопрос купили телят содержатся в одной клетке 5 голов на привязи, у одного телёнка сегодня с утра увидел что он пропоносился какойто прозрачной слизью с какимито белыми хлопьями (наверно овсянка которую ел) запах очень неприятный н...","[[341, 347], [372, 379], [180, 185], [357, 361]]"
97,391,"После отела у коровы ,не отошел послед и было очень твердое вымя.Лечили различными мазями,остались два уплотнения на передних долях вымени.Ни температуры ни сгустков не было.Спустя\n два месяца решили полечить,ввели септогель в доли. Теперь вымя твердое, доит мало, идут сгустки.","[[41, 45], [169, 173], [132, 138], [60, 64], [240, 244], [91, 95]]"
